## Configure PUDL
The `.pudl.yml` configuration file tells PUDL where to look for data. Uncomment the next cell and run it if you're on our 2i2c JupyterHub.

In [1]:
#!cp ~/shared/shared-pudl.yml ~/.pudl.yml

In [2]:
# import the necessary packages
%load_ext autoreload
%autoreload 2

import logging
import os
import sys

import pandas as pd
import sqlalchemy as sa
import pudl

In [3]:
# setup for python logging
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

## Set your EIA API Key
Some of the routines in this notebook pull data from the EIA API to fill in missing fuel cost values. For them to work, you'll need to provide an API KEY. You can [obtain one from EIA here](https://www.eia.gov/opendata/register.php), uncomment the code in the next cell, and use it to set the `API_KEY_EIA` environment variable to be your key. (If you're running this notebook on your own computer and want to set the environment variable outside of the notebook [see this blog post](https://www.twilio.com/blog/2017/01/how-to-set-environment-variables.html))

To set your API key in this notebook, uncomment the first line in the next cell and add your key in the quotes

In [4]:
# os.environ["API_KEY_EIA"] = "put your API key here"
assert os.environ.get("API_KEY_EIA") is not None
os.environ.get("API_KEY_EIA")

'017bec800b5c59369d5ef677c7839517'

# Using the PUDL output layer
The PUDL database tables are a clean, [normalized](https://en.wikipedia.org/wiki/Database_normalization) version of US electricity data. Normalized tables are great for databases and storage, but for interactive use, we often want a version of the data that includes plant and utility names and other associated info all in a single dataframe. These are "denormalized" tables. In addition to the referenced names and attributes like latitude and longitude or state, the denormalized tables often contain frequently calculated derived values (like calcuating `total_fuel_cost` from `total_heat_content_mmbtu` and `fuel_cost_per_mmbtu`). The Catalyst team developed a useful tool to access denormalized tables that we call the PUDL output object.

## What does the output layer provide?

Right now the output layer provides access to three different kinds of things:
 * denormalized tables
 * analytical outputs
 * partially integrated PUDL datasets that aren't in the database yet

## Why is the output layer useful?
Some benefits of using the output layer:
 * **Standardized denormalization:** You don't have to manually join the same tables together to get access to common attributes.
 * **Table caching:** many analyses rely on using the same table multiple times. The PUDL output object caches the tables in memory as pandas dataframes so you don't have to read tables from the database over and over again.
 * **Time series aggregation:** Some tables are annual, some monthly, some hourly. When you create a PUDL output object you can tell it to aggregate the data to either monthly or annual resolution for analysis.
 * **Standardized the filling-in of missing data:** There's a ton of missing or incomplete data. If requested, the output objects will use rolling averages and  data from the EIA API try to fill some of that missing data in.

# Instantiating Output Objects
* Grab the `pudl_settings`
* Create a connection to the PUDL Database
* Instantiate a `PudlTabl` object with that connection

In [5]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_settings

{'pudl_in': '/home/zane/code/catalyst/pudl-work',
 'data_dir': '/home/zane/code/catalyst/pudl-work/data',
 'settings_dir': '/home/zane/code/catalyst/pudl-work/settings',
 'pudl_out': '/home/zane/code/catalyst/pudl-work',
 'sqlite_dir': '/home/zane/code/catalyst/pudl-work/sqlite',
 'parquet_dir': '/home/zane/code/catalyst/pudl-work/parquet',
 'datapkg_dir': '/home/zane/code/catalyst/pudl-work/datapkg',
 'notebook_dir': '/home/zane/code/catalyst/pudl-work/notebook',
 'ferc1_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite'}

In [6]:
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_engine

Engine(sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite)

In [7]:
# this configuration will return tables without aggregating by a time frequency... we'll explore that more below.
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

## List the output object methods
* There are dozens of different data access methods within the `PudlTabl` object. If you want to see all of them with their docstrings, you can un-comment and run `help(pudl_out)` in the next cell.
* If you type `pudl_out.` and press `Shift` and `Tab` at the same time, you'll see a list of available methods as well.

In [8]:
#help(pudl_out)

This cell will print out a simple list of all the available public methods inside the `pudl_out` object

In [9]:
# this is the master list of all of the methods in the pudl_out object
# they all return a table cooresponding to their name
methods_pudl_out = [
    method_name for method_name in dir(pudl_out)
    if callable(getattr(pudl_out, method_name))    # if it is a method
    and '__' not in method_name                    # remove the internal methods
]
methods_pudl_out

['adjacency_ba_ferc714',
 'advanced_metering_infrastructure_eia861',
 'balancing_authority_assn_eia861',
 'balancing_authority_eia861',
 'bf_eia923',
 'bga',
 'bga_eia860',
 'capacity_factor',
 'demand_forecast_pa_ferc714',
 'demand_hourly_pa_ferc714',
 'demand_monthly_ba_ferc714',
 'demand_response_eia861',
 'demand_side_management_eia861',
 'description_pa_ferc714',
 'distributed_generation_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'energy_efficiency_eia861',
 'etl_eia861',
 'etl_ferc714',
 'fbp_ferc1',
 'frc_eia923',
 'fuel_cost',
 'fuel_ferc1',
 'gen_allocated_eia923',
 'gen_eia923',
 'gen_original_eia923',
 'gen_plants_ba_ferc714',
 'gens_eia860',
 'gf_eia923',
 'green_pricing_eia861',
 'hr_by_gen',
 'hr_by_unit',
 'id_certification_ferc714',
 'interchange_ba_ferc714',
 'lambda_description_ferc714',
 'lambda_hourly_ba_ferc714',
 'mcoe',
 'mergers_eia861',
 'net_energy_load_ba_ferc714',
 'net_metering_eia861',
 'non_net_metering_eia861',
 'operational_dat

## Basic Functionality

### Read a denormalized table
* Each of output object methods will return a Pandas Dataframe.
* Most of them correspond to a single database table, and will select all the data in that table, and automatically join it with some other useful information.
* Many of the access methods use an abbreviated name for the database table. E.g. the following reads all the data out of the `generators_eia860` table.

In [10]:
%%time
gens_eia860 = pudl_out.gens_eia860().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374180 entries, 374179 to 0
Columns: 110 entries, report_date to zip_code
dtypes: Int64(4), datetime64[ns](1), float64(21), int64(1), object(83)
memory usage: 318.3+ MB
CPU times: user 45 s, sys: 4.86 s, total: 49.8 s
Wall time: 1min


### Automatic dataframe caching
The `generators_eia860` table is quite long, and the above cell probably took several seconds to read 270,000 records each with 100 columns, creating an 800MB Dataframe. If you run the same output routine again, it will complete almost instantly because that dataframe is already stored inside `pudl_out`. This is memory intensive, but can save time in calculations that need to use the same tables several times.

In [11]:
%%time
gens_again_eia860 = pudl_out.gens_eia860()

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.06 µs


## Exploring `pudl_out` Arguments
Below, we'll explore the main arguments that are used to customize the PUDL output object. You can mix and match these options.

By default, the output object will read data from all available years, do no time aggregation, and not attempt to fill in missing values.

In [12]:
# here are the default arguments for the pudl_out object
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq=None,               # Desired time grouping to aggregate PUDL tables to.
    start_date=None,         # Beginning date for data to pull from the PUDL DB.
    end_date=None,           # End date for data to pull from the PUDL DB.
    fill_fuel_cost=False,    # Whether to fill in missing fuel costs with EIA monthly state-level averages.
    roll_fuel_cost=False,    # Whether to fill in monthly missing fuel costs with a 12-month rolling average.
)

### Time series aggregation
The PUDL output object can aggregate data on a monthly or annual basis, if you set the `freq` argument to `AS` (annual starting at the beginning of the calendar year) or `MS` (monthly starting at the beginning of the month) or [other equivalent frequency abbreviations](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases).

**NOTE:** Not all columns can be aggregated, so you may lose access to some kinds of information in aggregated outputs. If you need to retain information that gets lost in the default aggregation / groupby process, you may need to pull the unaggregated data and do your own aggregation.

In [13]:
pudl_out_as = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq='AS',               # Aggregate tables annually
)

In [14]:
pudl_out_as.gen_eia923().head()

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh
0,2009-01-01,3,32,Barry,195,18,Alabama Power Co,1,221908.0
1,2009-01-01,3,32,Barry,195,18,Alabama Power Co,2,394031.0
2,2009-01-01,3,32,Barry,195,18,Alabama Power Co,3,1286393.0
3,2009-01-01,3,32,Barry,195,18,Alabama Power Co,4,1626547.0
4,2009-01-01,3,32,Barry,195,18,Alabama Power Co,5,4513101.0


In [15]:
pudl_out_ms = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq='MS',               # Aggregate tables monthly
)

In [16]:
pudl_out_ms.gen_eia923().head()

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh
0,2009-01-01,3,32,Barry,195,18,Alabama Power Co,1,39699.0
1,2009-02-01,3,32,Barry,195,18,Alabama Power Co,1,5594.0
2,2009-03-01,3,32,Barry,195,18,Alabama Power Co,1,13015.0
3,2009-04-01,3,32,Barry,195,18,Alabama Power Co,1,15858.0
4,2009-05-01,3,32,Barry,195,18,Alabama Power Co,1,68232.0


### Filling in Missing Fuel Costs
 * The original EIA data is often incomplete.
 * Many utilities withold information about their fuel costs.
 * We have a couple of ways of estimating missing values, if you need complete data.

The ouput object created in the next cell will attempt to use all of these methods to fill in missing data.
To fill in missing fuel costs, we can pull monthly state-level average fuel costs from EIA, and we can use rolling averages to fill in short gaps in the data.
* Set `fill_fuel_cost=True` when creating an output object to pull average monthly fuel costs from the EIA API.
* Set `roll_fuel_cost=True` when creating an output object to use a 12-month rolling average based on available data to fill in gaps.
* These options can be used together to fill in as many gaps as possible.
* **NOTE:** You need to have set the `API_KEY_EIA` environment variable to a valid EIA API key for this to work. See instructions at the top of this notebook.

In [17]:
pudl_out_fill = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq='MS',               # Aggregate tables monthly
    fill_fuel_cost=True,     # Fill in missing fuel cost records with state-level averages from EIA's API
    roll_fuel_cost=True,     # Fill in missing fuel cost records with a 12-month rolling average.
)

In [18]:
%%time
pudl_out_fill.frc_eia923().head()

filling in fuel cost NaNs EIA APIs monthly state averages
filling in fuel cost NaNs with rolling averages
CPU times: user 2min 43s, sys: 2.59 s, total: 2min 45s
Wall time: 2min 59s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,ash_content_pct,chlorine_content_ppm,fuel_cost_from_eiaapi,fuel_cost_per_mmbtu,fuel_qty_units,fuel_type_code_pudl,heat_content_mmbtu_per_unit,mercury_content_ppm,moisture_content_pct,sulfur_content_pct,total_fuel_cost,total_heat_content_mmbtu
0,2009-01-01,3,32,Barry,195,18,Alabama Power Co,10.013475,NaN,False,4.526190,381438.0,coal,23.334763,NaN,NaN,0.938976,4.028655e+07,8900765.245
1,2009-02-01,3,32,Barry,195,18,Alabama Power Co,9.026785,NaN,False,4.096987,410147.0,coal,23.056621,NaN,NaN,0.822421,3.874359e+07,9456604.085
2,2009-03-01,3,32,Barry,195,18,Alabama Power Co,6.449671,NaN,False,3.709062,376787.0,coal,22.876760,NaN,NaN,0.487466,3.197087e+07,8619665.590
3,2009-04-01,3,32,Barry,195,18,Alabama Power Co,7.520152,NaN,False,3.897879,105322.0,coal,23.160458,NaN,NaN,0.523833,9.508120e+06,2439305.763
4,2009-05-01,3,32,Barry,195,18,Alabama Power Co,6.669016,NaN,False,3.672290,367333.0,coal,22.899997,NaN,NaN,0.621304,3.089103e+07,8411924.736


Looking at the filled vs. unfilled monthly data in the Fuel Receipts and Costs data from EIA 923, we can see that there are about 190k possible monthly records. Unfilled, we have fuel costs for about 107k of them. With the state level monthly fuel costs and rolling averages, we can get that up to about 116k records. An improvement, but it's not great. Unfortunately this data simply isn't reported publicly.

In [19]:
pudl_out_ms.frc_eia923()[["plant_id_eia", "report_date", "fuel_cost_per_mmbtu"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190115 entries, 0 to 190114
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   plant_id_eia         190115 non-null  Int64         
 1   report_date          190115 non-null  datetime64[ns]
 2   fuel_cost_per_mmbtu  106695 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(1)
memory usage: 6.0 MB


In [20]:
pudl_out_fill.frc_eia923()[["plant_id_eia", "report_date", "fuel_cost_per_mmbtu"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190115 entries, 0 to 190114
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   plant_id_eia         190115 non-null  Int64         
 1   report_date          190115 non-null  datetime64[ns]
 2   fuel_cost_per_mmbtu  115786 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(1)
memory usage: 6.0 MB


## Free Memory
Because this JupyterHub has limited memory, we need to delete the cached dataframes when we're done with them.

In [21]:
del pudl_out
del pudl_out_ms
del pudl_out_as
del pudl_out_fill

# Denormalized Output Tables
* Below, we'll extract and show a sample of many of the available denormalized PUDL output tables.
* If you'd like to see more than 5 sample rows, feel free to change `n_samples` below.
* Rather than assigning the results of these functions to a local variable in the notebook, we're showing samples from the cached dataframes to conserve memory, as this JupyterHub has limited RAM available at the moment.

In [22]:
n_samples = 5
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

## EIA Forms 860 & 923

In [23]:
# here are all of the EIA tables
tables_eia = [
    t for t in methods_pudl_out 
    if '_eia' in t 
    and '_eia861' not in t       # avoid the EIA 861 tables for now bc it is preliminary
]
tables_eia

['bf_eia923',
 'bga_eia860',
 'frc_eia923',
 'gen_allocated_eia923',
 'gen_eia923',
 'gen_original_eia923',
 'gens_eia860',
 'gf_eia923',
 'own_eia860',
 'plants_eia860',
 'pu_eia860',
 'utils_eia860']

### EIA Plant Utility Associations
This is mostly a helper function, used for adding plant and utility names and IDs into other output tables.

In [24]:
%%time
pudl_out.pu_eia860().sample(n_samples)

CPU times: user 7.73 s, sys: 39.7 ms, total: 7.77 s
Wall time: 7.99 s


,report_date,plant_id_eia,plant_name_eia,plant_id_pudl,utility_id_eia,utility_name_eia,utility_id_pudl
81238,2019-01-01,55816,Hawkeye Power Partners LLC,4691,8294,Hawkeye Power Partners LLC,1973
67150,2004-01-01,54413,Glenwood Springs Salt Project,14186,7301,Glenwood Springs Salt Co LP,<NA>
67006,2007-01-01,54391,Alice Falls Hydro Project,4084,326,"Alice Falls Hydro, LLC",444
25554,2013-01-01,3008,Mooreland,2237,20447,Western Farmers Elec Coop Inc,3737
56734,2016-01-01,50244,Canton North Carolina,3561,23815,Blue Ridge Paper Products Inc,656


### EIA 860 Boiler Generator Associations
* **NOTE:** We have filled in many more boiler-generator associations based on additional information. The `bga_source` column indicates where the association came from.

In [25]:
%%time
pudl_out.bga_eia860().sample(n_samples)

CPU times: user 772 ms, sys: 24 ms, total: 796 ms
Wall time: 799 ms


,plant_id_eia,report_date,generator_id,boiler_id,unit_id_eia,unit_id_pudl,bga_source
2991,562,2018-01-01,3,3,None,2,eia860_org
36354,10430,2009-01-01,GEN4,B9,None,1,eia860_org
13157,2070,2014-01-01,GTG2,GTG2,CC2,2,string_assn
34198,10328,2015-01-01,GEN4,BLR4,None,1,eia860_org
40706,10698,2014-01-01,GEN1,8,None,1,eia860_org


### EIA 860 Plants

In [26]:
%%time
pudl_out.plants_eia860().sample(n_samples)

CPU times: user 3.82 s, sys: 4.26 ms, total: 3.82 s
Wall time: 3.89 s


,plant_id_eia,plant_name_eia,balancing_authority_code_eia,balancing_authority_name_eia,city,county,ferc_cogen_status,ferc_exempt_wholesale_generator,ferc_small_power_producer,grid_voltage_kv,...,pipeline_notes,regulatory_status_code,transmission_distribution_owner_id,transmission_distribution_owner_name,transmission_distribution_owner_state,utility_id_eia,water_source,plant_id_pudl,utility_name_eia,utility_id_pudl
104905,58027,SunE CPS2 LLC,ERCO,"Electric Reliability Council of Texas, Inc.",San Antonio,Bexar,False,False,True,13.60,...,None,NR,16604,City of San Antonio - (TX),TX,57403,None,6195,SunE CPS2 LLC,3335
15901,1745,Trenton Channel,MISO,Midcontinent Independent Transmission System O...,Trenton,Wayne,False,False,False,120.00,...,None,RE,56068,None,MI,5109,Trenton Channel,590,DTE Electric Company,89
101543,57636,Auburn LFG Energy Facility,NYIS,New York Independent System Operator,Auburn,Cayuga,False,False,True,12.50,...,None,NR,13511,New York State Elec & Gas Corp,NY,56965,None,5872,"Auburn Energy, LLC",547
97764,57226,SPVP #12,CISO,California Independent System Operator,Ontario,None,False,False,False,0.48,...,None,RE,17609,Southern California Edison Co,CA,17609,None,5577,Southern California Edison Co,295
106978,58309,Porterville Solar,CISO,California Independent System Operator,Porterville,Tulare,False,False,True,12.47,...,None,NR,17609,Southern California Edison Co,CA,58268,None,6428,Tulare PV I LLC,3528


### EIA 860 Generators

In [27]:
%%time
pudl_out.gens_eia860().sample(n_samples)

CPU times: user 34.3 s, sys: 3.42 s, total: 37.7 s
Wall time: 38 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,associated_combined_heat_power,balancing_authority_code_eia,...,topping_bottoming_code,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code
255104,2009-01-01,55031,4350,Androscoggin Energy Center,55738,3662,Verso Paper Androscoggin LLC,CT01,True,ISNE,...,T,None,NaN,None,None,None,None,58.4,NaN,04239
235947,2010-01-01,50487,3677,PPG Powerhouse A,14306,1563,PPG Industries Inc,A2,False,MISO,...,X,None,0.0,None,None,None,None,7.5,NaN,70602
249375,2010-01-01,26,178,E C Gaston,<NA>,18,Alabama Power Co,2,False,SOCO,...,X,None,0.0,None,None,None,None,256.0,NaN,35186
290018,2008-01-01,1385,1795,Dale,5580,1571,East Kentucky Power Coop Inc,2,False,PJM,...,X,None,0.0,None,None,None,None,23.0,NaN,40391
45835,2018-01-01,7971,2986,Pella Peaking,14645,1148,Pella City of,1,False,MISO,...,X,None,NaN,None,None,False,None,NaN,NaN,50219


### EIA 860 Generator-level Ownership

In [28]:
%%time
pudl_out.own_eia860().sample(n_samples)

CPU times: user 7.72 s, sys: 80.4 ms, total: 7.8 s
Wall time: 7.8 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,owner_utility_id_eia,owner_name,fraction_owned,owner_city,owner_state,owner_street_address,owner_zip_code
36575,2013-01-01,50011,3457,Hawks Nest Hydro,55698,1974,Hawks Nest Hydro LLC,GEN3,55699,Brookfield Energy Marketing US LLC,1.00,Queensbury,NY,399 Big Bay Road399 Big Bay Road,12804
22819,2010-01-01,55570,<NA>,NaN,13644,<NA>,NaN,SR30,13644,Northern Alternative Energy,0.51,None,MN,None,None
60013,2018-01-01,60464,10765,Tenaska Westmoreland Generating Station,60249,5489,"Tenaska Pennsylvania Partners, LLC",CTG1,62017,"DGC Westmoreland, LLC",0.50,Omaha,NE,14302 FNB Parkway,68154
41583,2014-01-01,57845,6052,AC Landfill Energy LLC,57173,382,AC Landfill Energy LLC,UNIT2,56996,DCO Energy LLC,0.50,Mays Landing,NJ,5429 Harding Highway,08330
8385,2006-01-01,341,1402,Long Beach Generation LLC,13584,2673,NRG El Segundo Operations Inc,CT2,5504,Dynegy Inc,0.50,None,None,None,None


### EIA 923 Generation and Fuel Consumption

In [29]:
%%time
pudl_out.gf_eia923().sample(n_samples)

CPU times: user 22.5 s, sys: 1.47 s, total: 24 s
Wall time: 24.3 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,fuel_consumed_for_electricity_mmbtu,fuel_consumed_for_electricity_units,fuel_consumed_mmbtu,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type,fuel_type_code_aer,fuel_type_code_pudl,net_generation_mwh,nuclear_unit_id,prime_mover_code
919,2009-08-01,75,1321,Anchorage 1,599,498,Municipality of Anchorage,75425.0,73946.0,75425.0,73946.0,1.020,NG,NG,gas,6242.146,NaN,GT
982652,2016-09-01,50900,3852,Covington Facility,20508,2399,Westvaco Corp,1116.0,176.0,2689.0,424.0,6.342,RFO,RFO,oil,90.915,NaN,ST
73272,2009-01-01,50121,3499,Valero Refinery Corpus Christi West,19685,3643,Valero Refining Co,17665.0,17665.0,17665.0,17665.0,1.000,NG,NG,gas,1225.145,NaN,ST
1045403,2016-12-01,59579,7318,Franklin Solar 2,59324,1808,Franklin Solar 2 LLC,2920.0,0.0,2920.0,0.0,0.000,SUN,SUN,solar,316.279,NaN,PV
262992,2011-01-01,3114,2263,Shawnee (PA),17235,2690,RRI Energy Mid-Atlantic PH LLC,896.0,154.0,896.0,154.0,5.820,DFO,DFO,oil,24.424,NaN,GT


### EIA 923 Fuel Receipts and Costs

In [30]:
%%time
pudl_out.frc_eia923().sample(n_samples)

CPU times: user 20.4 s, sys: 349 ms, total: 20.8 s
Wall time: 20.8 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,ash_content_pct,chlorine_content_ppm,contract_expiration_date,...,moisture_content_pct,natural_gas_delivery_contract_type_code,natural_gas_transport_code,primary_transportation_mode_code,secondary_transportation_mode_code,state,sulfur_content_pct,supplier_name,total_fuel_cost,total_heat_content_mmbtu
299755,2015-11-01,10860,3432,Archer Daniels Midland Clinton,772,522,Archer Daniels Midland Co,5.1,NaN,2015-12-01,...,NaN,<NA>,<NA>,RR,<NA>,WY,0.22,arch,NaN,63927.500
370267,2017-07-01,8102,1131,"Gavin Power, LLC",61135,5533,"Gavin Power, LLC",8.2,0.0,2021-12-01,...,7.51,<NA>,<NA>,RV,<NA>,OH,4.17,consolidation coal company,NaN,715134.392
111029,2011-04-01,535,1519,McClellan (CA),16534,3069,Sacramento Municipal Util Dist,0.0,NaN,NaT,...,NaN,<NA>,firm,PL,<NA>,<NA>,0.00,edf trading,2.384000e+01,5.000
132587,2011-01-01,3942,1067,Albright,12796,191,Monongahela Power Co,11.9,NaN,2011-04-01,...,NaN,<NA>,<NA>,TR,<NA>,PA,1.10,gallatin,1.773883e+05,75709.920
327809,2016-06-01,8023,123,Columbia (WI),20856,364,Wisconsin Power & Light Co,5.1,0.0,2017-12-01,...,26.22,<NA>,<NA>,RR,<NA>,WY,0.25,arch coal sales,1.856555e+06,752861.074


### EIA 923 Boiler Fuel Consumption

In [31]:
%%time
pudl_out.bf_eia923().sample(n_samples)

CPU times: user 15.5 s, sys: 785 ms, total: 16.3 s
Wall time: 16.5 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,boiler_id,ash_content_pct,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type_code,fuel_type_code_pudl,sulfur_content_pct,total_heat_content_mmbtu
299677,2012-02-01,50202,3538,WPS Power Niagara,55807,2621,Niagara Generation LLC,1,0.0,0.0,0.00,TDF,waste,0.0,0.0
895718,2017-03-01,55464,4561,Deer Park Energy Center,4994,1447,Deer Park Energy Center,CTG6,0.0,1253010.0,1.03,NG,gas,0.0,1290600.3
656822,2015-03-01,54780,4243,University of Illinois Abbott Power Plt,19528,3585,University of Illinois,DB1,0.0,0.0,0.00,BIT,coal,0.0,0.0
21062,2009-03-01,2341,385,Mohave,17609,295,Southern California Edison Co,2,0.0,0.0,0.00,BIT,coal,0.0,0.0
583436,2015-09-01,2098,315,Lake Road (MO),56211,161,Evergy Missouri West,2,0.0,4116.0,1.00,NG,gas,0.0,4116.0


### EIA 923 Net Generation by Generator

In [32]:
%%time
pudl_out.gen_eia923().sample(n_samples)

CPU times: user 9.76 s, sys: 132 ms, total: 9.89 s
Wall time: 9.91 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh
337259,2017-12-01,862,1135,Grand Tower Energy Center LLC,59371,2341,Mainline Generation LLC,2,-685.0
397790,2018-03-01,6639,2691,R D Green,1692,40,Big Rivers Electric Corp,1,81097.0
472918,2019-11-01,57664,5896,Astoria Energy II,56991,540,Astoria Energy II LLC,CT3,63092.0
44362,2010-11-01,2080,391,Montrose,10000,159,Evergy Metro,2,58601.0
195159,2014-04-01,2850,288,J M Stuart,4922,317,Dayton Power & Light Co,1,277506.0


## FERC Form 1
* Only a small subset of the 100+ tables that exist in the original FERC Form 1 have been cleaned and included in the PUDL DB.
* For tables not included here, you'll need to access the cloned multi-year FERC 1 DB that we produce. See the first tutorial notebook for more information.

In [33]:
# All of the FERC Form 1 tables end with _ferc1
tables_ferc1 = [
    t for t in methods_pudl_out 
    if '_ferc1' in t 
]
tables_ferc1

['fbp_ferc1',
 'fuel_ferc1',
 'plant_in_service_ferc1',
 'plants_hydro_ferc1',
 'plants_pumped_storage_ferc1',
 'plants_small_ferc1',
 'plants_steam_ferc1',
 'pu_ferc1',
 'purchased_power_ferc1']

### FERC 1 Large Steam Plants
The large steam plants report detailed operating expenses in this table, as well as operational characteristics.

In [34]:
%%time
pudl_out.plants_steam_ferc1().sample(n_samples)

CPU times: user 697 ms, sys: 3.98 ms, total: 701 ms
Wall time: 706 ms


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_id_ferc1,plant_name_ferc1,asset_retirement_cost,avg_num_employees,capacity_factor,...,opex_steam,opex_steam_other,opex_structures,opex_transfer,peak_demand_mw,plant_capability_mw,plant_hours_connected_while_generating,plant_type,record_id,water_limited_capacity_mw
7327,1999,173,331,TXU Electric Company,8129,637,parkdale,NaN,13.0,0.128070,...,419672.0,NaN,266959.0,NaN,328.0,327.0,6770.0,steam,f1_steam_1999_12_173_2_3,NaN
19708,2005,159,292,South Carolina Electric & Gas Company,623,1374,urquhart #4,NaN,NaN,0.026345,...,NaN,NaN,NaN,NaN,54.0,NaN,308.0,combustion_turbine,f1_steam_2005_12_159_3_4,51.0
1091,2003,24,14,AEP Texas Central Company,1243,105,stp,73080391.0,NaN,0.682373,...,967304.0,NaN,1835114.0,NaN,653.0,630.0,12548.0,nuclear,f1_steam_2003_12_24_2_2,630.0
19645,2004,82,163,Kentucky Utilities Company,86,802,ew brown,3063030.0,125.0,0.584133,...,1815103.0,NaN,1322262.0,NaN,685.0,NaN,8064.0,steam,f1_steam_2004_12_82_0_3,NaN
22540,2006,288,339,"UNS Electric, Inc.",602,1482,valencia,NaN,4.0,0.003264,...,NaN,NaN,59277.0,NaN,59.0,61.0,115.0,combustion_turbine,f1_steam_2006_12_288_0_1,61.0


### FERC 1 Fuel
Fuel consumption by the large steam plants, broken down by plant and fuel type.

In [35]:
%%time
pudl_out.fuel_ferc1().sample(n_samples)

CPU times: user 397 ms, sys: 0 ns, total: 397 ms
Wall time: 400 ms


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_name_ferc1,fuel_consumed_mmbtu,fuel_consumed_total_cost,fuel_cost_per_mmbtu,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_mmbtu_per_unit,fuel_qty_burned,fuel_type_code_pudl,fuel_unit,record_id
11143,2008,62,130,Gulf Power Company,137,crist,5.620874e+07,1.919523e+08,3.415,79.460,88.719,23.268000,2415710.0,coal,ton,f1_fuel_2008_12_62_0_1
19891,2000,149,276,Public Service Electric and Gas Company,1167,keystone (steam),1.360541e+07,1.516282e+07,1.114,28.390,28.350,25.474000,534090.0,coal,ton,f1_fuel_2000_12_149_3_14
14564,2015,70,140,Idaho Power Company,294,jim bridger,3.682056e+04,5.639495e+05,15.316,90.059,108.869,5.880000,6262.0,oil,bbl,f1_fuel_2015_12_70_0_2
27223,2009,7,31,Arizona Public Service Company,739,west phoenix 1,1.443673e+06,1.455646e+07,10.083,10.281,3.808,1.019644,1415860.0,gas,mcf,f1_fuel_2009_12_7_6_4
24002,2002,281,150,Interstate Power and Light Company,219,george neal #3,2.154845e+04,9.367024e+04,4.350,4.360,4.360,1.003000,21484.0,gas,mcf,f1_fuel_2002_12_281_2_3


### FERC 1 Fuel by Plant
Wide-form aggregated fuel totals by plant and year, identifying the relative cost and heat content proportions of different fuels, as well as the primary fuel for the plant.

In [36]:
%%time
pudl_out.fbp_ferc1().sample(n_samples)

CPU times: user 502 ms, sys: 7.71 ms, total: 510 ms
Wall time: 516 ms


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_name_ferc1,coal_fraction_cost,coal_fraction_mmbtu,fuel_cost,fuel_mmbtu,...,nuclear_fraction_cost,nuclear_fraction_mmbtu,oil_fraction_cost,oil_fraction_mmbtu,primary_fuel_by_cost,primary_fuel_by_mmbtu,unknown_fraction_cost,unknown_fraction_mmbtu,waste_fraction_cost,waste_fraction_mmbtu
13777,2000,161,295,Southern California Edison Company,1230,san onofre unit 2&3,0.0,0.0,6.634700e+07,1.425787e+08,...,1.0,1.0,0.000000,0.000000,nuclear,nuclear,0.0,0.0,0.0,0.0
16401,2017,186,349,VIRGINIA ELECTRIC AND POWER COMPANY,617,warren,0.0,0.0,2.132289e+08,6.128905e+07,...,0.0,0.0,0.000000,0.000000,gas,gas,0.0,0.0,0.0,0.0
11810,2011,145,272,Public Service Company of Colorado,62,blue spruce,0.0,0.0,1.750177e+07,3.318398e+06,...,0.0,0.0,0.030852,0.011229,gas,gas,0.0,0.0,0.0,0.0
8563,2019,99,190,Mississippi Power Company,662,watson ct,0.0,0.0,2.660172e+05,6.727800e+04,...,0.0,0.0,0.000000,0.000000,gas,gas,0.0,0.0,0.0,0.0
721,2009,7,31,Arizona Public Service Company,570,sundance,0.0,0.0,1.642553e+07,1.741333e+06,...,0.0,0.0,0.000000,0.000000,gas,gas,0.0,0.0,0.0,0.0


### FERC 1 Plant in Service
An accounting of how much electric plant infrastructure exists in each of the many FERC accounts. This is a very wide form table.

In [37]:
%%time
pudl_out.plant_in_service_ferc1().sample(n_samples)

CPU times: user 1.04 s, sys: 8.02 ms, total: 1.04 s
Wall time: 1.05 s


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,record_id,amount_type,distribution_acct360_land,distribution_acct361_structures,distribution_acct362_station_equip,distribution_acct363_storage_battery_equip,...,transmission_acct352_structures,transmission_acct353_station_equip,transmission_acct354_towers,transmission_acct355_poles,transmission_acct356_overhead_conductors,transmission_acct357_underground_conduit,transmission_acct358_underground_conductors,transmission_acct359_1_asset_retirement,transmission_acct359_roads_trails,transmission_total
9406,2013,93,175,Massachusetts Electric Company,f1_plant_in_srvce_2013_12_93_0,additions,1180432.0,NaN,18231949.0,NaN,...,NaN,970231.0,NaN,77396.0,4877521.0,NaN,NaN,NaN,NaN,5925148.0
22100,2002,227,368,"Wolverine Power Supply Cooperative, Inc.",f1_plant_in_srvce_2002_12_227_0,ending_balance,1397219.0,210828.0,31967606.0,NaN,...,144021.0,32593705.0,3638020.0,19994002.0,19500382.0,NaN,55302.0,NaN,NaN,79155578.0
2237,1995,22,69,Cleco Power LLC,f1_plant_in_srvce_1995_12_22_0,starting_balance,7903930.0,59098.0,31448071.0,NaN,...,564854.0,135959315.0,11711942.0,52617297.0,45040884.0,NaN,128706.0,NaN,56362.0,265783785.0
19945,1995,189,15,AEP Texas North Company,f1_plant_in_srvce_1995_12_189_0,transfers,NaN,NaN,9643.0,NaN,...,NaN,744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,744.0
10355,2003,101,191,MONONGAHELA POWER COMPANY,f1_plant_in_srvce_2003_12_101_0,adjustments,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2922.0,NaN,2922.0


### FERC 1 Purchased Power
A summary of electricity market transactions between utilities. Sadly the sellers are identified only by their names, and not their FERC Utility (Respondent) ID.

In [38]:
%%time
pudl_out.purchased_power_ferc1().sample(n_samples)

CPU times: user 1.65 s, sys: 28 ms, total: 1.67 s
Wall time: 1.68 s


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,seller_name,record_id,billing_demand_mw,coincident_peak_demand_mw,delivered_mwh,demand_charges,energy_charges,non_coincident_peak_demand_mw,other_charges,purchase_type,purchased_mwh,received_mwh,tariff,total_settlement
126896,1999,67,138,Holyoke Water Power Company,Niagara Mohawk Energy Marketing Inc.,f1_purchased_pwr_1999_12_67_1_8,NaN,NaN,0.0,0.0,41836.0,NaN,0.0,other_service,1576.0,0.0,None,41836.0
16791,2009,21,57,Central Illinois Public Service Company,AmerenUE (Affiliate),f1_purchased_pwr_2009_12_21_2_4,NaN,NaN,0.0,862700.0,0.0,NaN,71944.0,other_service,0.0,0.0,None,934644.0
163735,2011,45,90,"Duke Energy Carolinas, LLC",Constellation Power Source Inc.,f1_purchased_pwr_2011_12_45_18_10,NaN,NaN,0.0,0.0,16557537.0,NaN,0.0,other_service,424773.0,0.0,Note (2),16557537.0
102203,1999,182,161,KCP&L Greater Missouri Operations Company,Missouri Basin d/b/a MO River Energy,f1_purchased_pwr_1999_12_182_2_14,NaN,NaN,0.0,0.0,411488.0,NaN,0.0,other_service,33192.0,0.0,MAPP-M,411488.0
106292,2017,186,349,VIRGINIA ELECTRIC AND POWER COMPANY,Total,f1_purchased_pwr_2017_12_186_0_15,NaN,NaN,0.0,6009156.0,494527805.0,NaN,0.0,None,12077869.0,0.0,None,500536961.0


## Free Memory
Again, because we're on a JupyterHub with limited RAM per user, we need to delete the cached dataframes we've just created.

In [39]:
del pudl_out

# Analysis Outputs
* The PUDL Database is mainly meant to standardize the structure of data that's been reported in different ways over different years, so that it can all be used together.
* We typically don't include calculated values or big modifications to the original data.
* We're compiling a growing library of stock analyses in the `pudl.analysis` subpackage, which operate on data stored in the database.
* Some of these analytical outputs are build into the output object so that they can take advantage of the dataframe caching, and for convenient access.

## The Marginal Cost of Electricity (MCOE)
* One of our first analysis modules calculates fuel costs, heat rates, and capacity factors on a generator by generator basis.
* The long term goal is for it to provide a comprehensive marginal cost of electricity production (MCOE).
* The integration of operating costs from FERC Form 1 is still a work in progress, and hasn't been added in here yet.

### MCOE Requires Aggregation
* Fuel costs and other data need to be aggregated by month or year to calculate MCOE.
* This means we need an output object that aggregates by month or year.
* Because a single `NA` value can wipe out a whole aggregated category, you'll get more information with a Monthly aggregation.

In [40]:
pudl_out_monthly = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    freq="MS",
    fill_fuel_cost=True,
    roll_fuel_cost=True,
)

### Heat Rate by Generation Unit (MMBTU/MWh)
* A "Generation Unit" (identifyed by `unit_id_pudl` here) is a group of "boilers" (where fuel is consumed) and "generators" (where electricity is made) which are connected to each other.
* Because the fuel inputs and electricity outputs are comingled, this is the most granular level at which a direct heat rate calculation can be done.

In [41]:
%%time
pudl_out_monthly.hr_by_unit().sample(n_samples)

CPU times: user 6min 14s, sys: 1.56 s, total: 6min 16s
Wall time: 6min 25s


,report_date,plant_id_eia,unit_id_pudl,net_generation_mwh,total_heat_content_mmbtu,heat_rate_mmbtu_mwh
129192,2013-07-01,10144,1,10680.0,222696.000,20.851685
199585,2015-12-01,55217,1,240474.0,1857534.216,7.724470
24863,2009-11-01,2952,4,58018.0,630954.000,10.875142
10549,2009-05-01,2951,3,53887.0,612079.160,11.358568
157864,2014-07-01,2171,2,0.0,0.000,NaN


### Heat Rate by Generator (mmBTU/MWh)
* However, we do need per-generator heat rates to estimate per-generator fuel costs.

In [42]:
%%time
pudl_out_monthly.hr_by_gen().sample(n_samples)

CPU times: user 36.7 s, sys: 3.17 s, total: 39.8 s
Wall time: 40.3 s


,report_date,plant_id_eia,heat_rate_mmbtu_mwh,generator_id,fuel_type_code_pudl,fuel_type_count
159764,2013-09-01,10670,NaN,GEN1,coal,2
243855,2015-04-01,7699,7.429376,CW1,gas,1
342067,2017-08-01,50191,NaN,TG3,waste,1
401384,2018-09-01,55661,7.479029,GT-2,gas,1
324871,2017-08-01,2001,30.883903,4,gas,2


### Per-generator Fuel Costs
* Calculate per-generator fuel costs based on heat rates and fuel deliveries
* Because we told the `pudl_out` object to try and fill in missing values, this will request monthly average fuel cost data by date from the EIA API. It might take a minute.
* This also means you'll need to have set your EIA API Key at the top of the notebook.

In [43]:
%%time
pudl_out_monthly.fuel_cost().sample(n_samples)

filling in fuel cost NaNs EIA APIs monthly state averages
filling in fuel cost NaNs with rolling averages
CPU times: user 2min 30s, sys: 1.28 s, total: 2min 31s
Wall time: 2min 38s


,plant_id_eia,report_date,generator_id,plant_name_eia,plant_id_pudl,utility_id_eia,utility_name_eia,utility_id_pudl,fuel_type_count,fuel_type_code_pudl,fuel_cost_from_eiaapi,fuel_cost_per_mmbtu,heat_rate_mmbtu_mwh,fuel_cost_per_mwh
144490,1745,2013-11-01,8,Trenton Channel,590,5109,The DTE Electric Company,89,1,coal,False,2.651292,15.582101,41.312694
30707,54429,2009-12-01,GEN2,Alabama Pine Pulp,4101,204,Alabama Pine Pulp Co Inc,4328,1,waste,NaN,NaN,33.616929,NaN
233821,2241,2015-02-01,3,C W Burdick,2033,40606,City of Grand Island,1013,1,gas,False,8.648000,-0.000000,-0.000000
55678,10511,2010-11-01,GEN6,Finch Pruyn,3283,56174,Finch Paper LLC,1721,1,oil,NaN,NaN,36.727435,NaN
358873,56350,2017-02-01,CT2A,Colorado Bend Energy Center,4959,56765,CER Colorado Bend Energy Partners LP,844,1,gas,False,NaN,8.422854,NaN


### Per-generator Capacity Factor

In [44]:
%%time
pudl_out_monthly.capacity_factor().sample(n_samples)

CPU times: user 500 ms, sys: 31 µs, total: 500 ms
Wall time: 499 ms


,plant_id_eia,report_date,generator_id,net_generation_mwh,capacity_mw,capacity_factor
40185,52006,2010-02-01,GEN4,10204.40,76.5,0.198498
475814,57908,2019-12-01,CTG2,8805.66,11.9,0.994585
123683,10075,2012-07-01,GEN2,0.00,84.0,0.000000
117097,3576,2012-05-01,3,5989.00,156.6,0.051403
450040,7512,2019-06-01,1,91659.00,187.5,0.678956


### Per-generator MCOE
* This function uses the cached dataframes that were generated above to produce a huge table of per-generator statistics.
* If you just called this function alone, all of those other dataframes would be automatically generated, and available within the output object.

In [45]:
%%time
pudl_out_monthly.mcoe().sample(n_samples)

CPU times: user 22.9 s, sys: 7.6 s, total: 30.5 s
Wall time: 32.8 s


,report_date,plant_id_eia,plant_id_pudl,unit_id_pudl,generator_id,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,associated_combined_heat_power,...,total_mmbtu,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code
233229,2015-10-01,2080,391,1.0,1,Montrose,10000,159,Evergy Metro,False,...,0.000000,None,NaN,None,None,False,None,170.0,NaN,64735
64313,2010-06-01,55131,4407,2.0,STG2,Kendall County Generation Facility,5517,1554,Dynegy Midwest Generation Inc,False,...,53.901000,None,NaN,None,None,None,None,125.0,NaN,60447
399417,2018-10-01,55380,4534,1.0,STG1,Union Power Station,814,106,Entergy Arkansas LLC,False,...,579692.947767,None,NaN,None,None,False,None,543.8,NaN,71730
373592,2018-09-01,3295,623,2.0,1,Urquhart,17539,292,"Dominion Energy South Carolina, Inc",False,...,291515.477296,None,NaN,None,None,False,None,65.0,NaN,29842
125573,2012-06-01,50006,3455,1.0,STG1,Linden Cogen Plant,3890,1597,EFS Cogen Holdings I LLC,True,...,35784.715156,None,NaN,None,None,None,None,100.0,NaN,07036


## Free Memory

In [46]:
del pudl_out_monthly

# Preliminary Output Tables 
* Integrating a new dataset into the PUDL database requires many steps (datastore, extract, transform, load, outputs).
* Sometimes we need to use tables from new datasets as soon as possible for analysis.
* The interim extract and transform steps can be hacked into the output object to run on the fly, prior to DB integration.
* The data extraction and transformation can take a while though -- and it will need to be re-run from scratch every time you create a new output object.
* **WARNING:** None of this data has been fully validated, and the structure is likely to change. Some of it (especially the FERC 714) is still in a pretty raw state.

As of December 2020, we have preliminarily integrated EIA 861 and FERC 714 in this format.

## EIA Form 861
* The interim EIA 861 ETL is set up to automatically run in its entirety as soon as you request any EIA 861 table.
* This should take 2-5 minutes if you already have the raw input data avaialble.
* If raw input data needs to be downloaded [from our Zenodo archives](https://zenodo.org/record/4127029) first (which should happen automatically), it will take longer.

In [47]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

In [48]:
# here are all of the EIA 861 tables
methods_eia861 = [t for t in methods_pudl_out if '_eia861' in t and "etl" not in t]
methods_eia861

['advanced_metering_infrastructure_eia861',
 'balancing_authority_assn_eia861',
 'balancing_authority_eia861',
 'demand_response_eia861',
 'demand_side_management_eia861',
 'distributed_generation_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'energy_efficiency_eia861',
 'green_pricing_eia861',
 'mergers_eia861',
 'net_metering_eia861',
 'non_net_metering_eia861',
 'operational_data_eia861',
 'reliability_eia861',
 'sales_eia861',
 'service_territory_eia861',
 'utility_assn_eia861',
 'utility_data_eia861']

### EIA 861 Balancing Authorities

In [49]:
%%time
pudl_out.balancing_authority_eia861().sample(n_samples)

Running the interim EIA 861 ETL process!
Extracting eia861 spreadsheet data.


/home/zane/code/catalyst/pudl/src/pudl/extract/eia861.py:35: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Transforming raw EIA 861 DataFrames for service_territory_eia861 concatenated across all years.
Assigned state FIPS codes fo

,report_date,balancing_authority_id_eia,balancing_authority_code_eia,balancing_authority_name_eia
14142,2005-01-01,9273,NaN,Indianapolis Power & Light Co
9216,2004-01-01,3755,NaN,Cleveland Electric Illum Co
3523,2002-01-01,7353,NaN,Golden Valley Elec Assn Inc
20948,2007-01-01,16463,NaN,City of Ruston
25402,2009-01-01,20847,NaN,Wisconsin Electric Power Co


### EIA 861 Advanced Metering Infrastructure

In [50]:
%%time
pudl_out.advanced_metering_infrastructure_eia861().sample(n_samples)

CPU times: user 5.62 ms, sys: 40 µs, total: 5.66 ms
Wall time: 5.28 ms


,utility_id_eia,state,balancing_authority_code_eia,report_date,entity_type,short_form,utility_name_eia,customer_class,advanced_metering_infrastructure,automated_meter_reading,daily_digital_access_customers,direct_load_control_customers,energy_served_ami_mwh,home_area_network,non_amr_ami
419,7129,TX,UNK,2007-01-01,NaN,<NA>,Georgetown City of,transportation,0,0,<NA>,<NA>,0.0,<NA>,<NA>
66574,40438,OR,BPAT,2015-01-01,NaN,<NA>,Columbia River Peoples Ut Dist,transportation,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
79665,2678,AR,MISO,2017-01-01,NaN,<NA>,C & L Electric Coop Corp,commercial,<NA>,2393,<NA>,83,NaN,<NA>,<NA>
21906,3634,MO,UNK,2011-01-01,NaN,<NA>,City of Nixa,industrial,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
77924,27000,NE,WACM,2016-01-01,NaN,<NA>,WAPA-- Western Area Power Administration,transportation,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>


### EIA 861 Sales
How much electricity did utilities report selling to different types of customers in each year by state?

In [51]:
%%time
pudl_out.sales_eia861().sample(n_samples)

CPU times: user 11.2 ms, sys: 0 ns, total: 11.2 ms
Wall time: 11 ms


,utility_id_eia,state,report_date,balancing_authority_code_eia,business_model,data_observed,entity_type,service_type,short_form,utility_name_eia,customer_class,customers,sales_mwh,sales_revenue
373040,58956,NH,2018-01-01,ISNE,retail,True,Retail Power Marketer,energy,<NA>,"SmartEnergy Holdings, LLC",other,NaN,NaN,NaN
9102,9998,MN,2001-01-01,UNK,retail,True,Municipal,bundled,<NA>,City of Kandiyohi,commercial,36.0,1161.0,60000.0
383389,7442,IA,2019-01-01,MISO,retail,True,Municipal,bundled,<NA>,Grafton Electric - (IA),industrial,NaN,NaN,NaN
370801,54862,DC,2018-01-01,PJM,retail,True,Retail Power Marketer,energy,<NA>,Champion Energy Services,industrial,0.0,0.0,0.0
222801,2056,KY,2011-01-01,UNK,retail,True,Municipal,bundled,<NA>,City of Bowling Green - (KY),residential,23128.0,250851.0,27194000.0


### EIA 861 Service Territories
Which counties (with FIPS codes) each utility reported serving in each year.

In [52]:
%%time
pudl_out.service_territory_eia861().sample(n_samples)

CPU times: user 9.57 ms, sys: 0 ns, total: 9.57 ms
Wall time: 11.1 ms


,county,short_form,state,utility_id_eia,utility_name_eia,report_date,state_id_fips,county_id_fips
74931,Palo Alto,<NA>,IA,9425,Iowa Lakes Electric Coop,2007-01-01,19,19147
23867,Alamosa,<NA>,CO,16603,"San Luis Valley R E C, Inc",2003-01-01,08,08003
54572,Spalding,<NA>,GA,7140,Georgia Power Co,2005-01-01,13,13255
221062,Sampson,<NA>,NC,17572,South River Elec Member Corp,2019-01-01,37,37163
170493,Carroll,<NA>,MS,12685,Entergy Mississippi Inc,2015-01-01,28,28015


### Free Memory

In [53]:
del pudl_out

## FERC Form 714
* **NOTE:** Most of the FERC Form 714 tables have not yet been fully processed.
* We have primarily been focused on the historical hourly demand reported by planning areas.
* As with the EIA 861, the full interim ETL will be run as soon as you ask for any FERC 714 table.
* Also as with the EIA 861, if you don't have the [raw FERC 714 input files](https://zenodo.org/record/4127101) cached locally already, they might take a minute to download.

In [54]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

In [55]:
# here are all of the FERC 714 tables
methods_ferc714 = [t for t in methods_pudl_out if '_ferc714' in t and "etl" not in t]
methods_ferc714

['adjacency_ba_ferc714',
 'demand_forecast_pa_ferc714',
 'demand_hourly_pa_ferc714',
 'demand_monthly_ba_ferc714',
 'description_pa_ferc714',
 'gen_plants_ba_ferc714',
 'id_certification_ferc714',
 'interchange_ba_ferc714',
 'lambda_description_ferc714',
 'lambda_hourly_ba_ferc714',
 'net_energy_load_ba_ferc714',
 'respondent_id_ferc714']

### FERC 714 Respondents
Currently the processing of the hourly planning area demand table exceeds the available memory on this JupyterHub, so the following cells are commented out.

In [56]:
%%time
respondent_id_ferc714 = pudl_out.respondent_id_ferc714()
respondent_id_ferc714.sample(5)

Running the interim FERC 714 ETL process!
Extracting respondent_id_ferc714 from CSV into pandas DataFrame.
Extracting id_certification_ferc714 from CSV into pandas DataFrame.
Extracting gen_plants_ba_ferc714 from CSV into pandas DataFrame.


/home/zane/code/catalyst/pudl/src/pudl/extract/ferc714.py:59: UserWarning: Integration of FERC 714 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


Extracting demand_monthly_ba_ferc714 from CSV into pandas DataFrame.
Extracting net_energy_load_ba_ferc714 from CSV into pandas DataFrame.
Extracting adjacency_ba_ferc714 from CSV into pandas DataFrame.
Extracting interchange_ba_ferc714 from CSV into pandas DataFrame.
Extracting lambda_hourly_ba_ferc714 from CSV into pandas DataFrame.
Extracting lambda_description_ferc714 from CSV into pandas DataFrame.
Extracting description_pa_ferc714 from CSV into pandas DataFrame.
Extracting demand_forecast_pa_ferc714 from CSV into pandas DataFrame.
Extracting demand_hourly_pa_ferc714 from CSV into pandas DataFrame.
Transforming respondent_id_ferc714.
Transforming id_certification_ferc714.
Transforming gen_plants_ba_ferc714.
Transforming demand_monthly_ba_ferc714.
Transforming net_energy_load_ba_ferc714.
Transforming adjacency_ba_ferc714.
Transforming interchange_ba_ferc714.
Transforming lambda_hourly_ba_ferc714.
Transforming lambda_description_ferc714.
Transforming description_pa_ferc714.
Transfor

,respondent_id_ferc714,respondent_name_ferc714,eia_code
114,223,Omaha Public Power District,14127
98,207,Montana-Dakota Utilities Company,12819
134,243,Sacramento Municipal Utility District,16534
138,247,Seattle City Light,16868
208,323,"Gridforce Energy Management, LLC",58790


### FERC 714 Hourly Demand by Planning Area

In [57]:
demand_hourly_pa_ferc714 = pudl_out.demand_hourly_pa_ferc714()
demand_hourly_pa_ferc714.sample(20)

,respondent_id_ferc714,report_date,utc_datetime,timezone,demand_mwh
9143095,139,2015-01-01,2015-07-09 22:00:00,America/Los_Angeles,610.50
7745568,194,2011-01-01,2011-01-20 20:00:00,America/Los_Angeles,3486.73
14879150,115,2019-01-01,2019-04-10 06:00:00,America/Denver,22.00
5836672,251,2010-01-01,2010-04-09 14:00:00,America/New_York,2619.00
3492,199,2006-01-01,2006-07-27 05:00:00,America/New_York,445.00
8678995,191,2019-01-01,2019-10-06 18:00:00,America/New_York,402.00
7005970,285,2009-01-01,2009-10-16 17:00:00,America/Chicago,511.00
7115761,163,2011-01-01,2011-07-30 17:00:00,America/Chicago,881.00
11644162,271,2016-01-01,2016-08-18 23:00:00,America/New_York,1282.00
984694,218,2013-01-01,2013-01-11 07:00:00,America/Chicago,2475.00


# Future Analyses
The output object contains a lot of different kinds of things, and as we accumulate more and different kinds of analyses in our library, we're looking to break them out into their own reusable classes that access the database directly. Some work in progress here is related to constructing historical service territory geometries for both utilities and balancing authorities, and associating that data usefully with the FERC 714 respondents. Exploring that in detail is beyond the scope of this notebook, but check out the `pudl.output.ferc714` and `pudl.analysis.service_territory` modules for examples. Unfortunately as this analysis currently depends on both the interim EIA 861 and the interim FERC 714 datasets, it uses too much memory to be run on the JupyterHub right now.

In [58]:
%%time
ferc714_out = pudl.output.ferc714.Respondents(pudl_out)
annualized = ferc714_out.annualize()
categorized = ferc714_out.categorize()
summarized = ferc714_out.summarize_demand()
fipsified = ferc714_out.fipsify()
counties_gdf = ferc714_out.georef_counties()

Running the interim EIA 861 ETL process!
Extracting eia861 spreadsheet data.


/home/zane/code/catalyst/pudl/src/pudl/extract/eia861.py:35: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Transforming raw EIA 861 DataFrames for service_territory_eia861 concatenated across all years.
Assigned state FIPS codes fo

/home/zane/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/missing.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


CPU times: user 2min 33s, sys: 2.35 s, total: 2min 36s
Wall time: 3min 7s
